In [12]:
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
matplotlib.use('TkAgg')  # or 'Qt5Agg', 'GTK3Agg', etc., depending on your system and available backends

from PIL import Image

def visualize_coco_tfrecord(tfrecord_path, num_images=5):
    raw_dataset = tf.data.TFRecordDataset(tfrecord_path)

    # Define feature description to parse the data
    feature_description = {
        'image/encoded': tf.io.FixedLenFeature([], tf.string),
        'image/object/bbox/xmin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/xmax': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymin': tf.io.VarLenFeature(tf.float32),
        'image/object/bbox/ymax': tf.io.VarLenFeature(tf.float32),
        'image/object/class/label': tf.io.VarLenFeature(tf.int64),
    }

    def _parse_function(example_proto):
        return tf.io.parse_single_example(example_proto, feature_description)

    parsed_dataset = raw_dataset.map(_parse_function)

    for idx, image_features in enumerate(parsed_dataset.take(num_images)):
        image = tf.image.decode_jpeg(image_features['image/encoded']).numpy()
        img_height, img_width, _ = image.shape

        fig, ax = plt.subplots(1)
        ax.imshow(image)

        xmin = tf.sparse.to_dense(image_features['image/object/bbox/xmin']).numpy()
        xmax = tf.sparse.to_dense(image_features['image/object/bbox/xmax']).numpy()
        ymin = tf.sparse.to_dense(image_features['image/object/bbox/ymin']).numpy()
        ymax = tf.sparse.to_dense(image_features['image/object/bbox/ymax']).numpy()

        for x_min, x_max, y_min, y_max in zip(xmin, xmax, ymin, ymax):
            x = x_min * img_width
            y = y_min * img_height
            width = (x_max - x_min) * img_width
            height = (y_max - y_min) * img_height

            rect = patches.Rectangle((x, y), width, height, linewidth=1, edgecolor='r', facecolor='none')
            ax.add_patch(rect)

        plt.show()

if __name__ == "__main__":
    tfrecord_path = "/home/yipeng/coco_tf_records/coco_train.record-00000-of-00100"
    visualize_coco_tfrecord(tfrecord_path)


2023-04-23 20:05:15.494524: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [1]
	 [[{{node Placeholder/_0}}]]


In [1]:
import tensorflow_datasets as tfds

(ds_train, ds_test), ds_info = tfds.load(
    'coco/2017',
    split=['train', 'validation'],
    shuffle_files=True,
    with_info=True,
    download=False
)

2023-04-23 19:58:30.282770: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-23 19:58:30.928837: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-04-23 19:58:31.491589: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-04-23 19:58:31.513499: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

Image shape: (480, 640, 3)
Labels: [23]


In [17]:
input_tensor = tf.keras.layers.Input(shape=(224, 224, 3), name="image")
model = tf.keras.applications.EfficientNetB0(
    input_tensor=input_tensor, weights=None, classes=91
)


model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()],
)



In [18]:

model.fit(
    ds_train,
    epochs=1,
    validation_data=ds_test,
)

2023-04-23 20:19:29.696496: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [256]
	 [[{{node Placeholder/_0}}]]
2023-04-23 20:19:29.696752: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int64 and shape [256]
	 [[{{node Placeholder/_4}}]]


ValueError: in user code:

    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/home/yipeng/miniconda3/envs/tf/lib/python3.9/site-packages/keras/engine/input